In [3]:
import pandas as pd
import requests
import os
import matplotlib.pyplot as plt
from IPython.display import display

In [8]:
# Make a GET request to download the JSON data
response = requests.get("https://www.onthesnow.com/_next/data/2.0.3_en-US/united-states/ski-resorts.json")

# Get the JSON content from the response
json_data = response.json()

# Extract the slug names of all resorts into a list
resorts_df = []
for resort in json_data["pageProps"]["regionResortListGuide"]:
    resort_slug = resort["slug"]
    resort_region = resort["region"]
    resorts_df.append({"Resort Slug": resort_slug, "Resort Region": resort_region})
    
regions_df = pd.DataFrame(resorts_df)

print(regions_df)

               Resort Slug Resort Region
0         49-degrees-north    washington
1               afton-alps     minnesota
2                 alpental    washington
3            alpine-valley          ohio
4     alpine-valley-resort     wisconsin
..                     ...           ...
326    wolf-creek-ski-area      colorado
327      woodbury-ski-area   connecticut
328  woods-valley-ski-area      new-york
329          yawgoo-valley  rhode-island
330            badger-pass    california

[331 rows x 2 columns]


In [9]:
#Making a df of Alta ski resort historic snowfall 
#This is a test for one resort. On The Snow makes frequent changes to API so ensure you are up to date

url = "https://www.onthesnow.com/_next/data/2.0.3_en-US/utah/alta-ski-area/historical-snowfall.json"
resort = url.split("/")[-2]

response = requests.get(url)
if response.status_code == 200:
    data = response.json()['pageProps']['snowfallInfoAnnual']
    df = pd.DataFrame(data)
    df.insert(0, "Resort", resort)
else:
    print("Failed to retrieve data from URL")
df

,Resort,date,totalSnow,snowDays,baseDepth,summitDepth,maxBaseDepth,biggestSnowfall
0,alta-ski-area,2012-01-01,929.64,76,127.805125,131.634525,287.02,43.18
1,alta-ski-area,2013-01-01,896.62,68,163.694388,153.623058,314.96,35.56
2,alta-ski-area,2014-01-01,706.12,56,111.931300,105.755465,228.60,48.26
3,alta-ski-area,2015-01-01,726.44,60,130.125275,128.836838,294.64,48.26
4,alta-ski-area,2016-01-01,772.16,55,162.523250,159.750743,368.30,48.26
5,alta-ski-area,2017-01-01,482.60,42,136.105825,130.396786,292.10,38.10
6,alta-ski-area,2018-01-01,1666.24,92,187.823625,185.848698,457.20,68.58
7,alta-ski-area,2019-01-01,828.04,53,127.134250,124.176069,340.36,60.96
8,alta-ski-area,2020-01-01,838.20,52,121.900537,126.027864,294.64,68.58
9,alta-ski-area,2021-01-01,779.78,57,144.692287,144.677069,289.56,60.96


In [16]:
# Create an empty list to store the links
links = []

# loop through each row and format the link, then append it to the list
for index, row in regions_df.iterrows():
    link = "https://www.onthesnow.com/_next/data/2.0.3_en-US/{}/{}/historical-snowfall.json".format(row['Resort Region'], row['Resort Slug'])
    links.append(link)

In [18]:
#Goes through each resort and gets data from 2012 to 2022 seasons and appends to DF
# WARNING - Can take a couple minutes to run!
dfs = []
for link in links:
    resort = link.split("/")[-2]
    response = requests.get(link)
    if response.status_code == 200:
        data = response.json()['pageProps']['snowfallInfoAnnual']
        df = pd.DataFrame(data)
        df['Resort'] = resort  
        dfs.append(df)
    else:
        print("Failed to retrieve data from URL")

result = pd.concat(dfs)

In [19]:
resort_df = result.merge(regions_df, left_on="Resort", right_on="Resort Slug", how="left")

# Drop the redundant "Resort Slug" column
resort_df.drop("Resort Slug", axis=1, inplace=True)

# Reorder columns (if needed)
resort_df = resort_df[['Resort', 'Resort Region', 'date', 'totalSnow', 'snowDays', 'baseDepth', 'summitDepth', 'maxBaseDepth', 'biggestSnowfall']]

In [20]:
# Convert the 'date' column to datetime format
resort_df['date'] = pd.to_datetime(resort_df['date'])

# Select only the year for each date
resort_df['date'] = resort_df['date'].dt.year

# Rename the 'date' column to 'year'
resort_df = resort_df.rename(columns={'date': 'year'})

In [22]:
# Convert 'totalSnow' from cm to inches
resort_df['totalSnow'] = resort_df['totalSnow'] * 0.393701

# Convert 'biggestSnowfall' from cm to inches
resort_df['biggestSnowfall'] = resort_df['biggestSnowfall'] * 0.393701

# Convert 'baseDepth' from cm to inches
resort_df['baseDepth'] = resort_df['baseDepth'] * 0.393701

# Convert 'summitDepth' from cm to inches
resort_df['summitDepth'] = resort_df['summitDepth'] * 0.393701

# Convert 'maxBaseDepth' from cm to inches
resort_df['maxBaseDepth'] = resort_df['maxBaseDepth'] * 0.393701

#Now all columns are converted from cm to inches

# Create a new column 'Avg Snow Day' by dividing 'totalSnow' by 'snowDays'
resort_df['Avg Snow Day'] = resort_df['totalSnow'] / resort_df['snowDays']


In [23]:
print(resort_df)

                Resort Resort Region  year   totalSnow  snowDays  baseDepth  \
0     49-degrees-north    washington  2012  313.000169        60  32.316991   
1     49-degrees-north    washington  2013  154.000083        55  23.739767   
2     49-degrees-north    washington  2014   57.000031        21   7.640550   
3     49-degrees-north    washington  2015  167.000090        49  42.685427   
4     49-degrees-north    washington  2016  283.000153        52  39.447743   
...                ...           ...   ...         ...       ...        ...   
3636       badger-pass    california  2018    0.000000         0   0.750000   
3637       badger-pass    california  2019   21.000011         3  11.250006   
3638       badger-pass    california  2020    0.000000         0   0.000000   
3639       badger-pass    california  2021   14.000008         4   0.000000   
3640       badger-pass    california  2022   78.000042         9   4.500002   

      summitDepth  maxBaseDepth  biggestSnowfall  A

In [24]:
#Making df a csv to use outside of Jupyter Notebook
resort_df.to_csv('Resort_Snowfall.csv', index=False)